In [ ]:
from datetime import datetime
from pymongo import MongoClient
import pandas as pd
import json
import copy

In [ ]:
def get_matches():
    client = MongoClient("mongodb://plp_mongodb:27017")
    db = client['plp']
    matches = list(db["match"].find({}))
    return matches
        

In [ ]:
def save_indicator_avg():
    data = pd.DataFrame(columns=["date", "hour", "league", "avg_indicator"])
    time = datetime.now().strftime("%d/%m/%Y-%H:%M").split("-")
    date = time[0]
    hour = time[1]

    
    profitable_matches = []
    for match in get_matches():
        if (match['ri_a'] > 0):
            match['side'] = "A"
            profitable_matches.append(copy.deepcopy(match))
        if (match['ri_d'] > 0):
            match['side'] = "D"
            profitable_matches.append(copy.deepcopy(match))
        if (match['ri_h'] > 0):
            match['side'] = "H"
            profitable_matches.append(copy.deepcopy(match))
    
    leagues = ['ligue1', 'premierLeague', 'liga', 'bundesliga', 'serieA', 'all']
    for i,league in enumerate(leagues) :
        sum = 0
        filter_matches = filter(lambda match: match['league'] == league or league == 'all', matches)
        for match in filter_matches:
            sum = sum + match['RI_' + match['side']]
        avg_indicator = sum/len(filter_matches)
        data.loc[i] = [date, hour, league, avg_indicator]
        
    return data

In [ ]:
def hash_ind(date, hour, league):
    date = date.split("/")
    hour = hour.split(":")
    return date[0]+date[1]+date[2]+hour[0]+hour[1]+league

In [ ]:
def update_db(data):
    client = MongoClient("mongodb://plp_mongodb:27017")
    coll = client["plp"]["iterator"]

    for index, row in data.iterrows():
        coll.replace_one({'_id':row['_id']}, row.to_dict(), True)

In [ ]:
def main_indicator():
    data = save_indicator_avg()
    
    ids_ind = pd.DataFrame()
    ids_ind["_id"] = data.iloc[:,[0,1,2]].apply(lambda row: hash_ind(row.iloc[0],row.iloc[1],row.iloc[2]), axis = 1)
    data.insert(0, '_id',  ids_ind["_id"])
    
    update_db(data)

In [ ]:
main_indicator()